In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
df = pd.read_csv("../FIFA 2006-2018 - Cleaned.csv")

In [3]:
df.head()

,Team1,Team2,Rounds,Stadium,City,Day,Month,Year,Score1,Score2,Remarks
0,Russia,Saudi Arabia,Group Phase,Luzhniki Stadium,Moscow,14,6,2018,5,0,NaN
1,Egypt,Uruguay,Group Phase,Ekaterinburg Arena,Ekaterinburg,15,6,2018,0,1,NaN
2,Morocco,IR Iran,Group Phase,Saint Petersburg Stadium,St. Petersburg,15,6,2018,0,1,NaN
3,Portugal,Spain,Group Phase,Fisht Stadium,Sochi,15,6,2018,3,3,{WinTeamName} win on aggregate after regular t...
4,France,Australia,Group Phase,Kazan Arena,Kazan,16,6,2018,2,1,NaN


In [4]:
df = df.drop(["Remarks","Stadium","City"],axis=1)
countries = list(set(df["Team1"]).union(set(df["Team2"])))
countries_dict = {}
for i in range(len(countries)):
    countries_dict[countries[i]] = i
    
Team1_encoded = []
Team2_encoded = []

for i in range(len(df)):
    Team1_encoded.append(countries_dict[list(df.Team1)[i]])
    Team2_encoded.append(countries_dict[list(df.Team2)[i]])
df["Team1"] = Team1_encoded
df["Team2"] = Team2_encoded

In [5]:
df.head()

,Team1,Team2,Rounds,Day,Month,Year,Score1,Score2
0,33,13,Group Phase,14,6,2018,5,0
1,3,35,Group Phase,15,6,2018,0,1
2,53,16,Group Phase,15,6,2018,0,1
3,54,9,Group Phase,15,6,2018,3,3
4,37,6,Group Phase,16,6,2018,2,1


In [6]:
rounds = df.Rounds
rounds.unique()
rounds_dict = {"Group Phase":1, "Round of 16":2, "Quarter Finals":3, "Semi Finals":4, "Play-off for third place":5, "Finals":6}

rounds_encoded = []

for i in range(len(df)):
    rounds_encoded.append(rounds_dict[list(df.Rounds)[i]])
df["Rounds"] = rounds_encoded

winning_team = []

for i in range(len(df)):
    if df.Score1[i]>df.Score2[i]:
        winning_team.append(1)
    elif df.Score1[i]<df.Score2[i]:
        winning_team.append(2)
    else:
        winning_team.append(0)
        
df['winning_team'] = winning_team

In [9]:

df.head()

,Team1,Team2,Rounds,Day,Month,Year,Score1,Score2,winning_team
0,33,13,1,14,6,2018,5,0,1
1,3,35,1,15,6,2018,0,1,2
2,53,16,1,15,6,2018,0,1,2
3,54,9,1,15,6,2018,3,3,0
4,37,6,1,16,6,2018,2,1,1


In [19]:
# df['avg_team1']=[]
avg_team1=[]
avg_team2=[]

for index_1,row_1 in df.iterrows():

    if row_1['Year'] == 2018:
        avg_team1.append(row_1['Score1']/91)
        avg_team2.append(row_1['Score2']/78)
        

    if row_1['Year'] == 2014:
        avg_team1.append(row_1['Score1']/81)
        avg_team2.append(row_1['Score2']/90)
        
        
    if row_1['Year'] == 2010:
        avg_team1.append(row_1['Score1']/76)
        avg_team2.append(row_1['Score2']/69)

    if row_1['Year'] == 2006:
        avg_team1.append(row_1['Score1']/86)
        avg_team2.append(row_1['Score2']/61)

print(len(avg_team1)," ",len(avg_team2))
df['avg_team1'] = avg_team1
df['avg_team2'] = avg_team2

256   256


In [20]:
df.head()

,Team1,Team2,Rounds,Day,Month,Year,Score1,Score2,winning_team,avg_team1,avg_team2
0,33,13,1,14,6,2018,5,0,1,0.054945,0.000000
1,3,35,1,15,6,2018,0,1,2,0.000000,0.012821
2,53,16,1,15,6,2018,0,1,2,0.000000,0.012821
3,54,9,1,15,6,2018,3,3,0,0.032967,0.038462
4,37,6,1,16,6,2018,2,1,1,0.021978,0.012821


In [21]:
for index,row in df.iterrows():
    df.set_value(index+255,["Team1","Team2","Rounds","Day","Month","Year","Score1","Score2","winning_team","avg_team1","avg_team2"],[row["Team2"],row["Team1"],row["Rounds"],row["Day"],row["Month"],row["Year"],row["Score2"],row["Score1"],2 if row["winning_team"]==1 else 1 if row["winning_team"]==2 else 0,row['avg_team2'],row['avg_team1']])
df.shape


/home/milad/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


(511, 11)

In [ ]:
df.head(10)

In [ ]:
# for index, row in df.iterrows():
# df[df['Team1']==41]
df.groupby(['Year'])['Team1'].sum()

In [22]:
y=df['winning_team']
df=df.drop(['winning_team'], axis=1)
# X=df

In [23]:
X = pd.get_dummies(df,prefix=['Team1','Team2'],columns=['Team1','Team2'])




In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [26]:
model = Sequential()
model.add(Dense(12, input_dim=X.shape[1], activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
#model.dropout()
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #categorical crossentropy?
model.fit(X_train, y_train, validation_data=(X_test,y_test), shuffle=True, epochs=100, batch_size=64)


Train on 459 samples, validate on 52 samples
Epoch 1/100
459/459 [==============================] - 0s 779us/step - loss: 0.8662 - acc: 0.2222 - val_loss: 0.8241 - val_acc: 0.2115
Epoch 2/100
459/459 [==============================] - 0s 61us/step - loss: 0.7854 - acc: 0.2222 - val_loss: 0.7400 - val_acc: 0.2115
Epoch 3/100
459/459 [==============================] - 0s 81us/step - loss: 0.7089 - acc: 0.2680 - val_loss: 0.6603 - val_acc: 0.3654
Epoch 4/100
459/459 [==============================] - 0s 89us/step - loss: 0.6358 - acc: 0.3922 - val_loss: 0.5890 - val_acc: 0.3654
Epoch 5/100
459/459 [==============================] - 0s 62us/step - loss: 0.5714 - acc: 0.3922 - val_loss: 0.5231 - val_acc: 0.3654
Epoch 6/100
459/459 [==============================] - 0s 65us/step - loss: 0.5127 - acc: 0.3922 - val_loss: 0.4605 - val_acc: 0.3654
Epoch 7/100
459/459 [==============================] - 0s 69us/step - loss: 0.4563 - acc: 0.3922 - val_loss: 0.4036 - val_acc: 0.3654
Epoch 8/100
459/

Epoch 61/100
459/459 [==============================] - 0s 64us/step - loss: -0.4145 - acc: 0.3922 - val_loss: -0.5582 - val_acc: 0.3654
Epoch 62/100
459/459 [==============================] - 0s 72us/step - loss: -0.4214 - acc: 0.3922 - val_loss: -0.5663 - val_acc: 0.3654
Epoch 63/100
459/459 [==============================] - 0s 65us/step - loss: -0.4282 - acc: 0.3922 - val_loss: -0.5735 - val_acc: 0.3654
Epoch 64/100
459/459 [==============================] - 0s 57us/step - loss: -0.4339 - acc: 0.3922 - val_loss: -0.5808 - val_acc: 0.3654
Epoch 65/100
459/459 [==============================] - 0s 74us/step - loss: -0.4399 - acc: 0.3922 - val_loss: -0.5881 - val_acc: 0.3654
Epoch 66/100
459/459 [==============================] - 0s 58us/step - loss: -0.4457 - acc: 0.3922 - val_loss: -0.5954 - val_acc: 0.3654
Epoch 67/100
459/459 [==============================] - 0s 61us/step - loss: -0.4514 - acc: 0.3922 - val_loss: -0.6028 - val_acc: 0.3654
Epoch 68/100
459/459 [===================

In [27]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

52/52 [==============================] - 0s 119us/step

acc: 36.54%


In [25]:
from sklearn import svm
from sklearn.metrics import accuracy_score
model_svm =svm.SVC()

model_svm.fit(X_train,y_train)
y_pred_svm = model_svm.predict(X_test)

print(accuracy_score(y_test, y_pred_svm))


0.903846153846


In [28]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model_inf = ExtraTreesClassifier()
model_inf.fit(X_train, y_train)

# model_tree= DecisionTreeClassifier()
# model_tree.fit(X_train,y_train)

# y_pred_tree = model_tree.predict(X_test)

# print(accuracy_score(y_test, y_pred_tree))


information_gain = {}

for i in range(len(model_inf.feature_importances_)):
    information_gain.update({i: model_inf.feature_importances_[i]})
    
    
for key in information_gain: 
    print(key,information_gain[key])

0 0.0111965647877
1 0.0180112038309
2 0.00628512075938
3 0.0218925285033
4 0.197951419554
5 0.106513254677
6 0.122483730354
7 0.229280779388
8 0.00162304927332
9 0.0111442116527
10 0.00320249538058
11 0.000743158459933
12 0.00196367972807
13 0.0037701823182
14 0.00381397002882
15 0.0016928992274
16 0.00235496775757
17 0.0046073851256
18 0.00923201215173
19 0.00116468413808
20 0.000623948069696
21 0.0019993230895
22 0.000820145482267
23 0.0
24 0.00158080941771
25 0.00149657553708
26 0.000600296034085
27 0.000864668544323
28 0.00600122460549
29 0.00378388784211
30 0.00372133807303
31 0.00183414554057
32 0.000764964332975
33 0.00332359546762
34 0.00149618826765
35 0.000823298721406
36 0.000302725656997
37 0.000266161658871
38 0.000880326336835
39 0.000529199604102
40 0.00471589732846
41 0.00650148772151
42 0.0030197053151
43 0.00365732663721
44 0.00349371877736
45 0.00712613144354
46 0.00462366317178
47 0.00279699463466
48 0.0012247738833
49 5.51438740537e-05
50 0.00206492626991
51 0.0013